In [ ]:
rdata_read = load("../../../data/wage2015_subsample_inference.RData")
data = rdata_read["data"]
names(data)
println("Number of Rows : ", size(data)[1],"\n","Number of Columns : ", size(data)[2],) #rows and columns

# An inferential problem: The Gender Wage Gap

In the previous lab, we already analyzed data from the March Supplement of the U.S. Current Population Survey (2015) and answered the question how to use job-relevant characteristics, such as education and experience, to best predict wages. Now, we focus on the following inference question:

What is the difference in predicted wages between men and women with the same job-relevant characteristics?

Thus, we analyze if there is a difference in the payment of men and women (*gender wage gap*). The gender wage gap may partly reflect *discrimination* against women in the labor market or may partly reflect a *selection effect*, namely that women are relatively more likely to take on occupations that pay somewhat less (for example, school teaching).

To investigate the gender wage gap, we consider the following log-linear regression model

\begin{align}
\log(Y) &= \beta'X + \epsilon\\
&= \beta_1 D  + \beta_2' W + \epsilon,
\end{align}

where $D$ is the indicator of being female ($1$ if female and $0$ otherwise) and the
$W$'s are controls explaining variation in wages. Considering transformed wages by the logarithm, we are analyzing the relative difference in the payment of men and women.

In [ ]:
Pkg.add("DataFrames")
Pkg.add("Dates")
Pkg.add("Plots")
Pkg.add("CategoricalArrays")

In [ ]:
using Pkg
using DataFrames
using Dates
using Plots
using Statistics,RData  #upload data of R format 
using CategoricalArrays # categorical data 

***Variable description***

- occ : occupational classification
- ind : industry classification
- lwage : log hourly wage
- sex : gender (1 female) (0 male)
- shs : some high school
- hsg : High school graduated
- scl : Some College
- clg: College Graduate
- ad: Advanced Degree
- ne: Northeast
- mw: Midwest
- so: South
- we: West
- exp1: experience

### **1.  Analysis in the subset of workers with advanced college education (variables scl, clg, ad).**

Next, we will conduct an analysis for the subset of workers with advanced college education. To do this, we will restrict our data and keep only those who have Some College, College Graduate, or Advanced Degree. 

In [ ]:
sub_data = data[(data[!, :scl] .== 1) .| (data[!, :clg] .== 1) .| (data[!, :ad] .== 1), :]
Z = data[(data[!, :scl] .== 1) .| (data[!, :clg] .== 1) .| (data[!, :ad] .== 1), 
         [:lwage, :sex, :shs, :hsg, :scl, :clg, :ad, :ne, :mw, :so, :we, :exp1]]
data_female = data[(data[!, :sex] .== 1) .& ((data[!, :scl] .== 1) .| (data[!, :clg] .== 1) .| (data[!, :ad] .== 1)), :]
Z_female = data_female[!, [:lwage, :sex, :shs, :hsg, :scl, :clg, :ad, :ne, :mw, :so, :we, :exp1]]

# Filtrar datos para hombres
data_male = data[(data[!, :sex] .== 0) .& ((data[!, :scl] .== 1) .| (data[!, :clg] .== 1) .| (data[!, :ad] .== 1)), :]
Z_male = data_male[!, [:lwage, :sex, :shs, :hsg, :scl, :clg, :ad, :ne, :mw, :so, :we, :exp1]]

means = DataFrame( variables = names(Z), All = describe(Z, :mean)[!,2], Men = describe(Z_male,:mean)[!,2], Female = describe(Z_female,:mean)[!,2])


In particular, the table above shows that the difference in average logwage between men and women is equal to  $0.0750$

In [ ]:
mean(Z_female[:,:lwage]) - mean(Z_male[:,:lwage])

Thus, the unconditional gender wage gap is about $7,5$\% for the group of never married workers (women get paid less on average in our sample). 

#### 1.1 OLS estimation without controls

In [ ]:
#install all the package that we can need
#Pkg.add("GLM") # package to run models 
#Pkg.add("StatsPlots")
#Pkg.add("MLBase")
#Pkg.add("Tables")
#Pkg.add("CovarianceMatrices") # robust standar error 
# Load the installed packages
using DataFrames
using Tables
using GLM
using CovarianceMatrices


In [ ]:
nocontrol_model = lm(@formula(lwage ~ sex), sub_data)
nocontrol_est = GLM.coef(nocontrol_model)[2]
nocontrol_se = GLM.coeftable(nocontrol_model).cols[2][2]

# nocontrol_se1 = stderror(HC1(), nocontrol_model)[2]
CI1upper = confint(nocontrol_model)[2, 2]
CI1low = confint(nocontrol_model)[2, 1]

println("The estimated gender coefficient is ", nocontrol_est ," and the corresponding robust standard error is " ,nocontrol_se)

#### 1.2 OLS estimation with controls

Next, we run an ols regression of $Y$ on $(D,W)$ to control for the effect of covariates summarized in $W$:

\begin{align}
\log(Y) &=\beta_1 D  + \beta_2' W + \epsilon.
\end{align}

Here, we are considering the flexible model from the previous lab. Hence, $W$ controls for experience, education, region, and occupation and industry indicators plus transformations and two-way interactions.

In [ ]:
flex = @formula(lwage ~ sex + (exp1+exp2+exp3+exp4) * (shs+hsg+scl+clg+occ2+ind2+mw+so+we))
control_model = lm(flex , sub_data)
control_est = GLM.coef(control_model)[2]
control_se = GLM.coeftable(control_model).cols[2][2]
#control_se1 = stderror( HC0(), control_model)[2]
CI2upper = confint(control_model)[2, 2]
CI2low = confint(control_model)[2, 1]

println("Coefficient for OLS with controls " , control_est, "robust standard error:", control_se)

#### 1.3 Partialling-Out using ols

In [ ]:
# models
# model for Y
flex_y = @formula(lwage ~ (exp1+exp2+exp3+exp4) * (shs+hsg+scl+clg+occ2+ind2+mw+so+we))
flex_d = @formula(sex ~ (exp1+exp2+exp3+exp4) * (shs+hsg+scl+clg+occ2+ind2+mw+so+we))

t_Y = residuals(lm(flex_y, sub_data))
t_D = residuals(lm(flex_d, sub_data))

data_res = DataFrame(t_Y = t_Y, t_D = t_D )
partial_fit = lm(@formula(t_Y ~ t_D), data_res)
partial_est = GLM.coef(partial_fit)[2]

# standard error
partial_se = GLM.coeftable(partial_fit).cols[2][2]

#partial_se1 = stderror( HC0(), partial_fit)[2]

#condifence interval
CI3upper = confint(partial_fit)[2, 2]
CI3low = confint(partial_fit)[2, 1]
println("Coefficient for D via partiallig-out ", partial_est, " robust standard error:", partial_se )

We know that the partialling-out approach works well when the dimension of $W$ is low
in relation to the sample size $n$. When the dimension of $W$ is relatively high, we need to use variable selection
or penalization for regularization purposes. 

In the following, we illustrate the partialling-out approach using lasso instead of ols. 

In [ ]:
DataFrame(modelos = [ "Without controls", "full reg", "partial reg" ], 
Estimate = [nocontrol_est,control_est, partial_est], 
StdError = [nocontrol_se,control_se, partial_se])

### **2.   Descriptive statistics subset of workers with advanced college education (variables scl, clg, ad).**


##### 2.1)  Wage & Lwage

In [ ]:
plot_density = density(data.wage, fillcolor = :blue, fillalpha = 0.5, 
                        legend = false, title = "Figure1: Density of wages",
                        xlabel = "Log Wage", ylabel = "Density")

In [ ]:
plot_density = density(data.lwage, fillcolor = :blue, fillalpha = 0.5, 
                        legend = false, title = "Figure1: Density of wages",
                        xlabel = "Log Wage", ylabel = "Density")

#### 2.2 Descriptive data by gender

In [ ]:
Bplot = boxplot(data.lwage, title = "BoxPlot - Log Wage by Gender",
    ylabel = "Log Wage", xlabel = "Gender",label = "All", legend =:outerright )
boxplot!(Z_female.lwage, label = "Female" )
boxplot!(Z_male.lwage, label = "Male")

### **3. Confidence Interval of sex's coefficient for a different models:**


In [ ]:
CIdf = DataFrame(
    x = ["No Control", "With Control", "Partialling Out"],
    y = [nocontrol_est, control_est, partial_est],
    lower = [CI1low, CI2low, CI3low],
    upper = [CI1upper, CI2upper, CI3upper]
)

In [ ]:
using Gadfly
plot_density =  Gadfly.plot(CIdf, x=:x, y=:y, ymin=:lower, ymax=:upper,
                    Geom.point(), Geom.errorbar(),
                    Guide.title("Figure 9: C.I. of the sex variable according to different estimations."),
                    Theme(panel_fill="white", panel_stroke=colorant"black", default_color=colorant"darkred"))
display(plot_density)


### **4. Replication of the figure: Experience Profiles and Wage Gap for High School Graduates**


In [ ]:
data_hsg = data[data.hsg .== 1, :]
data_clg = data[data.clg .== 1, :]
data_scl = data[data.scl .== 1, :]

data_clgm = data_clg[data_clg.sex .== 0, :]  # Hombres
data_clgf = data_clg[data_clg.sex .== 1, :]  # Mujeres

In [ ]:
# Tabla_hsg
Tabla_hsg = data_hsg.groupby('exp1').agg(Promlwageo=('lwage', 'mean')).reset_index()
nivel_hsg = sorted(data_hsg['exp1'].unique())

Promedio = []
for nivel in nivel_hsg:
    Promedio.append(data_hsg[data_hsg['exp2'] <= nivel]['lwage'].mean())

Tabla_hsg['PromMov'] = Promedio
print(Tabla_hsg.head())

# Tabla_clg
Tabla_clg = data_clg.groupby('exp2').agg(Promlwageo=('lwage', 'mean')).reset_index()
Tabla_clgm = data_clg.groupby('exp2').agg(Promlwageo=('lwage', 'mean')).reset_index()
Tabla_clgf = data_clg.groupby('exp2').agg(Promlwageo=('lwage', 'mean')).reset_index()

nivel_clg = sorted(data_clg['exp2'].unique())
nivel_clgm = sorted(data_clgm['exp2'].unique())
nivel_clgf = sorted(data_clgf['exp2'].unique())

Promedio = []
for nivel in nivel_clg:
    Promedio.append(data_clg[data_clg['exp2'] <= nivel]['lwage'].mean())

Tabla_clg['PromMov'] = Promedio
Tabla_clgm['PromMov'] = Promedio
Tabla_clgf['PromMov'] = Promedio
print(Tabla_clg.head())


In [ ]:
# Define the equation
formula = @formula(lwage ~ sex + (exp1 + exp2 + exp3 + exp4) * (shs + hsg + scl + clg + occ2 + ind2 + mw + so + we))

control_fit1 = lm(formula, data)


predict = predict(control_fit1)

data[!, :Predict] = predict


data_sclP = filter(row -> row.scl == 1, data)
data_clgP = filter(row -> row.clg == 1, data)
data_hsgP = filter(row -> row.hsg == 1, data)

data_clgPm = filter(row -> row.clg == 1 && row.sex == 0, data)  # Men
data_clgPf = filter(row -> row.clg == 1 && row.sex == 1, data)  # Women


In [ ]:
using Statistics


Tabla_hsgP = by(data_hsgP, :exp2, Predict = :Predict => mean)
Tabla_hsgP = sort!(Tabla_hsgP, :exp2)

nivel_hsgP = sort(unique(data_hsgP.exp2))

Promedio = [mean(data_hsgP[data_hsgP.exp2 .<= nivel, :Predict]) for nivel in nivel_hsgP]
Tabla_hsgP[!, :PromMovP] = Promedio

println(first(Tabla_hsgP, 5))


Tabla_clgP = by(data_clgP, :exp2, Predict = :Predict => mean)
Tabla_clgP = sort!(Tabla_clgP, :exp2)

Tabla_clgPf = by(data_clgPf, :exp2, Predict = :Predict => mean)
Tabla_clgPf = sort!(Tabla_clgPf, :exp2)

Tabla_clgPm = by(data_clgPm, :exp2, Predict = :Predict => mean)
Tabla_clgPm = sort!(Tabla_clgPm, :exp2)

nivel_clgP = sort(unique(data_clgP.exp2))

Promedio = [mean(data_clgP[data_clgP.exp2 .<= nivel, :Predict]) for nivel in nivel_clgP]
Tabla_clgP[!, :PromMov] = Promedio
Tabla_clgPf[!, :PromMov] = Promedio

Promediof = Promedio[1:end-1]
Tabla_clgPm[!, :PromMov] = Promediof

println(first(Tabla_clgP, 5))

In [ ]:
using Plots

x = Tabla_clg[:exp2]
x_3 = Tabla_clgPm[:exp2]
y = Tabla_clg[:PromMov]
y_3 = Tabla_clgPm[:PromMov]

# Creathe the graphic
plot(x, y, color=:navy, linestyle=:solid, label="Actual CLG")
plot!(x_3, y_3, color=:darkred, linestyle=:dash)


ylims!(3, 3.2)
xlims!(0, 15)
xlabel!("Years of Potential Experience")
ylabel!("Log Wage (or Wage Gap)")
title!("Comparison between actual and fitted for CLG and HSG Male")
grid!(linestyle=:dash, color=:gray)


xticks!(0:5:15)


legend!(:topright, fontsize=8)


display(plot!())


x = Tabla_clg[:exp2]
x_3 = Tabla_clgPf[:exp2]
y = Tabla_clg[:PromMov]
y_3 = Tabla_clgPf[:PromMov]


plot(x, y, color=:navy, linestyle=:solid, label="Actual CLG")
plot!(x_3, y_3, color=:darkred, linestyle=:dash)


ylims!(3, 3.2)
xlims!(0, 15)
xlabel!("Years of Potential Experience")
ylabel!("Log Wage (or Wage Gap)")
title!("Comparison between actual and fitted for CLG and HSG Female")
grid!(linestyle=:dash, color=:gray)


xticks!(0:5:15)


legend!(:topright, fontsize=8)


display(plot!())